# 🤝 Interactions

Before you start:
- Make sure you have the following packages installed via `pip`:
    - TODO ~~`energyplus-ooep`~~
    - `energyplus-dataset`

What you will learn in this chapter:
- How to _control_ simulator lifecycles.
- How to _access_ simulator variables
- How to _subscribe_ to simulator events.
- How to _visualize_ simulator variables.

TODO

## 🧳 Prepare

In [1]:
from controllables.core import TemporaryUnavailableError
from controllables.energyplus import World, Actuator, OutputMeter, OutputVariable
from controllables.energyplus.logging import ProgressLogger, MessageLogger, VariableLogger

from energyplus.dataset.basic import dataset as _epds_

## 💨 Run!

In [2]:
world = World(
    input=World.Specs.Input(
        world=(
            _epds_.models / '1ZoneEvapCooler.idf'
        ),
        weather=(_epds_.weathers / 'USA_CO_Denver-Aurora-Buckley.AFB.724695_TMY3.epw'),
    ),
    # TODO
    #runtime=dict(recurring=True),
).add(ProgressLogger()).add(MessageLogger())

  0%|          | 0/100 [00:00<?, ?it/s]

In [3]:
_ = world.awaitable.run()
#world.run()

In [4]:
#world.stop()

## 👀 Observe

TODO introduce manager pattern; references

### 🔣 Variables

In [5]:
list(world.variables.keys())

[]

### 🔔 Events

In [6]:
list(world.events.keys())

[Ref(name='message', include_warmup=True),
 Ref(name='progress', include_warmup=True)]

Keep me updated - Get [in the "loop"](https://wiktionary.org/wiki/in_the_loop), [literally](https://wikipedia.org/wiki/Event_loop).

In [7]:
_ = '''
world.events.on('message', lambda event: print('🥳', event))

@world.on('message')
def _(event): print('😋', event)

@world.events['message'].use
def _(event): print('😎', event)
'''

### 💲 Data

TODO

In [8]:
world.variables

In [9]:
try:
    print(world['wallclock'].value)
    print(
        world[OutputVariable.Ref(
            type='Site Outdoor Air Drybulb Temperature', 
            key='ENVIRONMENT',
        )].value
    )
    print(
        world.variables[
            Actuator.Ref(
                type='Weather Data', 
                control_type='Environment', 
                key='Outdoor Dry Bulb',
            )
        ].value        
    )
except TemporaryUnavailableError:
    pass

2003-03-13 12:50:00
22.833333333333336


In [10]:
_ = world.add(
    vlogger := VariableLogger(
        {'maxlen': 5_000},
        autoupdate='begin_zone_timestep_after_init_heat_balance',
    )
)

In [11]:
vlogger.plot({
    'traces': [
        # {
        #     'x': 'wallclock:calendar',
        #     'y': Actuator.Ref(
        #         type='Zone Temperature Control',
        #         control_type='Heating Setpoint',
        #         key='CORE_MID',
        #     ),
        #     'label': 'Temperature Setpoint',
        # },
        {
            'x': 'wallclock:calendar',
            # 'y': OutputVariable.Ref(
            #     type='People Air Temperature',
            #     key='CORE_MID',
            # ),
            # 'label': 'Mean Temperature',
            'y': world[OutputVariable.Ref('Site Outdoor Air Drybulb Temperature', 'ENVIRONMENT')] 
                - world[OutputVariable.Ref('Zone Mean Air Temperature', 'MAIN ZONE')],            
            'label': 'Air Temperature Diff',
        },
    ],
}, autoupdate=1_000)

In [12]:
#vlogger.dataframe()

In [13]:
#import itables
#itables.show(vlogger.dataframe())

## 🤔 Reflect

### 🎩 Tricks

In [14]:
world.variables.available_keys()